In [23]:
import urllib3
import certifi
import re
import pandas as pd
from IPython.display import display, HTML
import pymongo
import json

url = 'https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2014&sort=desc&view=detailed'



**Connect to Metacritic and retrieve HTML**

In [24]:

#Grab a connection from the connection pool
http = urllib3.PoolManager(ca_certs=certifi.where())

#Fixes bombing error in VS Code.  Thanks to Jamie Kirsila for the fix here.  This is not needed in Google Colab, for whatever reason.
http.headers={'User-Agent': 'Mozilla/5.0'}

#Retrieve the webpage
resp = http.request('GET', url)

#Grab the html of the webpage
datastring = str(resp.data, "utf-8")

Get connection string from file for connecting to MongoDB

In [25]:

#Method from https://www.geeksforgeeks.org/read-json-file-using-python/

# Opening JSON file
json_file = open('C:\\Users\\stuar\\Downloads\\MongoDBConnection.json')
  
# returns JSON object as 
# a dictionary
data = json.load(json_file)

# Grab connection info from JSON file
uri = data['mongodb']

# Closing file
json_file.close() 

Connect to MongoDB

In [41]:
#Method from https://www.geeksforgeeks.org/mongodb-and-python/?ref=gcse

#Connect to db
client = pymongo.MongoClient(uri)

# Access database
mydatabase = client['metacritic2']
  
# Access collection of the database
mycollection = mydatabase['movies']


In [27]:
#Make an array of the regular expressions
regex = ['','','','','']
regex[0] = '><h3>(.*)<\/h'                        #Title
regex[1] = 's">\s+<span>(.*)<'                    #Release Date  
regex[2] = '>(.*)<\/div>\s+<\/a>\s+<\/div>\s+<s'  #Score
regex[3] = 'e-wrap.*\n.*<img src="(.*)" a'        #Thumbnail URL
regex[4] = 'y">\s+(.*)\s+<\/d'                    #Description

In [28]:
#Make an array of the set of matches for each regex.  This includes extra data we don't want for this assignment, so this won't be our final stop.
#The line of code used here was adapted from the code generator on RegEx101
matches = []
for x in range(5):
    matches.append(re.finditer(regex[x], datastring, re.MULTILINE))


In [29]:
#Instantiate an array for each field
Title = []
ReleaseDate = []
CriticScore = []
ThumbnailURL = []
Description = []

In [30]:
#Using a Dataframe just to try to refresh my skills from my Python class.  It has a lot of useful features like easy sorting, if you need them.
df = pd.DataFrame()
df.insert(0,"Title", [])
df.insert(1,"Release Date", [])
df.insert(2,"Critic Score", [])
df.insert(3,"Thumbnail URL", [])
df.insert(4,"Description", [])

In [31]:
#Strip out just the text from the matches and put it in an array.  
#The line of code used here was adapted from the code generator on RegEx101
for matchNum, match in enumerate(matches[0], start=1):
    Title.append(match.group(1))
for matchNum, match in enumerate(matches[1], start=1):
    ReleaseDate.append(match.group(1))
for matchNum, match in enumerate(matches[2], start=1):
    CriticScore.append(match.group(1))
for matchNum, match in enumerate(matches[3], start=1):
    ThumbnailURL.append(match.group(1))
for matchNum, match in enumerate(matches[4], start=1):
    Description.append(match.group(1))

In [32]:
#Make each array a column in the dataframe
df = pd.DataFrame()
df.insert(0,"Title", Title)
df.insert(1,"Release Date", ReleaseDate)
df.insert(2,"Critic Score", CriticScore)
df.insert(3,"Thumbnail URL", ThumbnailURL)
df.insert(4,"Description", Description)

In [49]:
for index, row in df.iterrows():    
    mydatabase['movies'].insert_one(row.to_dict)
    print(row.to_dict)
    #movies_json = movie.to_json()
    #print(movie)

<bound method Series.to_dict of Title                                                      Boyhood
Release Date                                         July 11, 2014
Critic Score                                                   100
Thumbnail URL    https://static.metacritic.com/images/products/...
Description      Filmed over 12 years with the same cast, Richa...
Name: 0, dtype: object>
<bound method Series.to_dict of Title                                                   Mr. Turner
Release Date                                     December 19, 2014
Critic Score                                                    94
Thumbnail URL    https://static.metacritic.com/images/products/...
Description      Mr. Turner explores the last quarter century o...
Name: 1, dtype: object>
<bound method Series.to_dict of Title                                                    Leviathan
Release Date                                     December 25, 2014
Critic Score                                        